In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score
wordnet_lemmatizer = WordNetLemmatizer()

In [2]:
df = pd.read_csv('Tweets.csv')

In [3]:
df.shape

(14640, 15)

In [4]:
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [5]:
def normalizer(tweet):
      only_letters = re.sub("[^a-zA-Z]", " ", tweet)
      only_letters = only_letters.lower()
      only_letters = only_letters.split()
      filtered_result = [word for word in only_letters if word not in stopwords.words('english')]
      lemmas = [wordnet_lemmatizer.lemmatize(t) for t in filtered_result]
      lemmas = ' '.join(lemmas)
      return lemmas

In [6]:
df = shuffle(df)
y = df['airline_sentiment']
x = df.text.apply(normalizer)

In [7]:
vectorizer = CountVectorizer()
x_vectorized = vectorizer.fit_transform(x)

In [8]:
train_x,val_x,train_y,val_y = train_test_split(x_vectorized,y)

In [9]:
regressor = LogisticRegression(multi_class='multinomial', solver='newton-cg')
model = regressor.fit(train_x, train_y)

In [10]:
params = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] }
gs_clf = GridSearchCV(model, params, n_jobs=1, cv=5)
gs_clf = gs_clf.fit(train_x, train_y)
model = gs_clf.best_estimator_

In [11]:
y_pred = model.predict(val_x)

_f1 = f1_score(val_y, y_pred, average='micro')
_confusion = confusion_matrix(val_y, y_pred)
__precision = precision_score(val_y, y_pred, average='micro')
_recall = recall_score(val_y, y_pred, average='micro')
_statistics = {'f1_score': _f1,
               'confusion_matrix': _confusion,
               'precision': __precision,
               'recall': _recall
              }

In [12]:
print(_statistics)

{'f1_score': 0.7879781420765027, 'confusion_matrix': array([[2063,  174,   72],
       [ 262,  424,   70],
       [ 105,   93,  397]], dtype=int64), 'precision': 0.7879781420765027, 'recall': 0.7879781420765027}


In [13]:
test_feature = vectorizer.transform(['Meat Week Day 3: Tummy hurts every night'])
model.predict(test_feature)

array(['negative'], dtype=object)

In [14]:
test_feature = vectorizer.transform(['Movie is good'])
model.predict(test_feature)

array(['positive'], dtype=object)

In [15]:
test_feature = vectorizer.transform(['I\'m okay'])
model.predict(test_feature)

array(['neutral'], dtype=object)

In [16]:
import pickle

In [17]:
pickl = {'vectorizer': vectorizer,
         'model': model
         }
pickle.dump(pickl, open('models.pkl', "wb"))